<a href="https://colab.research.google.com/github/sebasrosalesr/Sample-projects/blob/main/Marging_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>







# Margin automated Overview:

This program is designed to **automatically adjust pricing** based on **margins** provided in a CSV file. Specifically, it calculates a **new sell price** for products where the margin is below 20%. If the margin is already 20% or higher, the sell price remains unchanged.

---

### Key Steps in the Program:
1. **Load the CSV Data:**
   - The program reads in a CSV file that contains product data, including columns for **Sell Price**, **New Cost**, **Base U of M Qty**, and **Margin on price/cost**.

2. **Dynamic Column Identification:**
   - It dynamically finds the correct columns using keywords like "Sell", "Margin on price/cost", and "New Cost ?", ensuring flexibility even if there are slight variations in the column names.

3. **Margin Calculation:**
   - For each product, the margin is converted from a percentage (e.g., `-41.17%`) to a decimal, and if the margin is **below 20%**, a **new sell price** is calculated using the formula:

     \[
     \text{New Price} = \frac{\text{New Cost}}{1 - \text{Target Margin}}
     \]

   - If the margin is **above 20%**, the original sell price is kept.

4. **Status Column:**
   - An additional column is added to track whether the price has been **updated** or **left unchanged**.

5. **Export Results:**
   - The updated pricing data is saved to a new **Excel file** with the calculated prices, ensuring that any changes are clearly captured and ready for review.

---

### How It Works:
1. The program loads the CSV data.
2. For each row, it checks the margin:
   - If the margin is less than 20%, the new price is calculated.
   - If it’s above 20%, the original price is retained.
3. It writes the results to a new Excel file, including a "Status" column to track updates.

---

### Use Case:
This program is ideal for pricing teams who need to adjust product prices dynamically to meet specific margin targets. It automates the calculation and ensures consistency across large datasets.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:

#Creating data frame.(Doesn't matter if the typos of the columns has subtle differences the code will fix it)
df= pd.read_csv('/content/drive/MyDrive/Pricing porcentage sheet /MGM test CSV.csv')
df.columns.value_counts()


,count
Doc Date,1
SOP Number,1
Cust Number,1
Cust Class ID,1
Customer Name,1
Corp Assoc,1
Item Number,1
Item Description,1
Manufacturer,1
Part #,1


In [ ]:
# **Finding Column Indices for Calculations**

# The following section aims to identify the indices of columns
# that will be crucial in calculating the percentage increase
# required to meet the desired margin. This involves searching
# for specific keywords within the column names of the DataFrame.

def find_column_index(df, keywords):
    """
    Function to find the index of the column that contains the given keywords.

    Parameters:
    df (pd.DataFrame): The DataFrame to search.
    keywords (list of str): The keywords to search for.

    Returns:
    int: The index of the column that contains the keywords, or -1 if not found.
    """
    for col in df.columns:
        if all(keyword in col for keyword in keywords):
            return df.columns.get_loc(col)
    return -1

# Example usage
keywords = ['Base U of M Qty']
column_index = find_column_index(df, keywords)

if column_index != -1:
    print(f"The index of the column containing the keywords '{keywords}' is: {column_index}")
else:
    print(f"No column found containing the keywords '{keywords}'")


keywords_margingcost = ['Margin on price/cost']
column_index_sexo = find_column_index(df, keywords_margingcost)

if column_index_sexo != -1:
    print(f"The index of the column containing the keyword '{keywords_margingcost}' is: {column_index_sexo}")
else:
    print(f"No column found containing the keyword '{keywords_margingcost}'")

keywords_sell_price = ['Sell']
column_index_sexo = find_column_index(df, keywords_sell_price)

if column_index_sexo != -1:
    print(f"The index of the column containing the keyword '{keywords_sell_price}' is: {column_index_sexo}")
else:
    print(f"No column found containing the keyword '{keywords_sell_price}'")


keywords_new_cost = ['New Cost ?']
column_index_sexo = find_column_index(df, keywords_new_cost)

if column_index_sexo != -1:
    print(f"The index of the column containing the keyword '{keywords_new_cost}' is: {column_index_sexo}")
else:
    print(f"No column found containing the keyword '{keywords_new_cost}'")



The index of the column containing the keywords '['Base U of M Qty']' is: 11
The index of the column containing the keyword '['Margin on price/cost']' is: 18
The index of the column containing the keyword '['Sell']' is: 19
The index of the column containing the keyword '['New Cost ?']' is: 22


In [ ]:
def get_price_for_margin(cost, target_margin):
    return cost / (1 - target_margin)

# Example usage
new_cost = 1.25
target_margin = 0.2
new_price = get_price_for_margin(new_cost, target_margin)
print(f"New price to achieve {target_margin*100}% margin: ${new_price:.4f}")

New price to achieve 20.0% margin: $1.5625


In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Pricing porcentage sheet /MGM test CSV.csv')

# Define keyword-based column finder
def find_column_index(df, keywords):
    for col in df.columns:
        if all(keyword.lower() in col.lower() for keyword in keywords):
            return df.columns.get_loc(col)
    return -1

# Find necessary column indices
idx_buom = find_column_index(df, ['Base U of M Qty'])
idx_margin = find_column_index(df, ['Margin on price/cost'])
idx_sell_price = find_column_index(df, ['Sell'])
idx_new_cost = find_column_index(df, ['New Cost ?'])

# Define margin calculation and price update logic
def get_price_for_margin(cost, target_margin=0.2):
    try:
        return cost / (1 - target_margin)
    except ZeroDivisionError:
        return 0

# Add new column for updated prices (you can name it as you like)
new_price_col_name = "Updated Sell Price for 20% Margin"
df[new_price_col_name] = None  # Initialize column

for index, row in df.iterrows():
    try:
        margin = float(str(row.iloc[idx_margin]).strip().replace('%', '')) / 100
        sell_price = float(str(row.iloc[idx_sell_price]).replace('$', '').replace(',', '').strip())
        new_cost = float(str(row.iloc[idx_new_cost]).replace('$', '').replace(',', '').strip())

        if margin < 0.2:
            new_price = round(get_price_for_margin(new_cost), 4)
        else:
            new_price = sell_price

        df.at[index, new_price_col_name] = new_price

    except Exception as e:
        print(f"Error processing row {index}: {e}")
        df.at[index, new_price_col_name] = 'ERROR'


In [ ]:
df.head()             # Shows the first 5 rows
df.head(10)

,Doc Date,SOP Number,Cust Number,Cust Class ID,Customer Name,Corp Assoc,Item Number,Item Description,Manufacturer,Part #,...,New Sell 2,New Extended,New Cost ?,New Extended Cost,New Margin,Change,NH contract used as cost in file for some accounts such as Legacy,Unnamed: 27,Unnamed: 28,Updated Sell Price for 20% Margin
0,01/21/25,INV13372888,MGM49,STANDARD,Camelot Nursing and Rehabilitation Center,Midwest Geriatric,1008646,Glucerna Shake Vanilla Plastic Bottle 8oz - 24...,ABBOTT NUTRITION,57801,...,NaN,NaN,$1.25,NaN,NaN,NaN,NaN,NaN,NaN,1.5625
1,12/17/24,INV13230545,MGM49,STANDARD,Camelot Nursing and Rehabilitation Center,Midwest Geriatric,1008646,Glucerna Shake Vanilla Plastic Bottle 8oz - 24...,ABBOTT NUTRITION,57801,...,NaN,NaN,$1.25,NaN,NaN,NaN,NaN,NaN,NaN,1.5625
2,12/26/24,INV13267986,MGM49,STANDARD,Camelot Nursing and Rehabilitation Center,Midwest Geriatric,1008646,Glucerna Shake Vanilla Plastic Bottle 8oz - 24...,ABBOTT NUTRITION,57801,...,NaN,NaN,$1.25,NaN,NaN,NaN,NaN,NaN,NaN,1.5625
3,12/09/24,INV13192256,MGM45,STANDARD,Florissant Valley Health & Rehabilitation Center,Midwest Geriatric,1008646,Glucerna Shake Vanilla Plastic Bottle 8oz - 24...,ABBOTT NUTRITION,57801,...,NaN,NaN,$1.25,NaN,NaN,NaN,NaN,NaN,NaN,1.5625
4,01/08/25,INV13313626,MGM39,STANDARD,Sunset Health Care Center,Midwest Geriatric,1008646,Glucerna Shake Vanilla Plastic Bottle 8oz - 24...,ABBOTT NUTRITION,57801,...,NaN,NaN,$1.25,NaN,NaN,NaN,NaN,NaN,NaN,1.5625
5,12/26/24,INV13267985,MGM45,STANDARD,Florissant Valley Health & Rehabilitation Center,Midwest Geriatric,1008646,Glucerna Shake Vanilla Plastic Bottle 8oz - 24...,ABBOTT NUTRITION,57801,...,NaN,NaN,$1.25,NaN,NaN,NaN,NaN,NaN,NaN,1.5625
6,01/15/25,INV13348129,MGM45,STANDARD,Florissant Valley Health & Rehabilitation Center,Midwest Geriatric,1008646,Glucerna Shake Vanilla Plastic Bottle 8oz - 24...,ABBOTT NUTRITION,57801,...,NaN,NaN,$1.25,NaN,NaN,NaN,NaN,NaN,NaN,1.5625
7,12/17/24,INV13232528,MGM49,STANDARD,Camelot Nursing and Rehabilitation Center,Midwest Geriatric,1008646,Glucerna Shake Vanilla Plastic Bottle 8oz - 24...,ABBOTT NUTRITION,57801,...,NaN,NaN,$1.25,NaN,NaN,NaN,NaN,NaN,NaN,1.5625
8,01/16/25,INV13354840,MGM49,STANDARD,Camelot Nursing and Rehabilitation Center,Midwest Geriatric,1008646,Glucerna Shake Vanilla Plastic Bottle 8oz - 24...,ABBOTT NUTRITION,57801,...,NaN,NaN,$1.25,NaN,NaN,NaN,NaN,NaN,NaN,1.5625
9,01/06/25,INV13300592,MGM18,STANDARD,Oak Park Nursing Center,Midwest Geriatric,1008646,Glucerna Shake Vanilla Plastic Bottle 8oz - 24...,ABBOTT NUTRITION,57801,...,NaN,NaN,$1.25,NaN,NaN,NaN,NaN,NaN,NaN,1.5625


In [ ]:
import os

folder_path = '/content/drive/MyDrive/Pricing porcentage sheet'
os.makedirs(folder_path, exist_ok=True)

In [ ]:
output_path = f'{folder_path}/Updated_MGM_Pricing.xlsx'
df.to_excel(output_path, index=False)